# Weight Notebook

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/briggs_imaging_weights.ipynb)

The purpose of this notebook is to demonstrate that there is agreement between CASA and AstroVIPER when calculating imaging weights using Briggs weighting with robust values of [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2].  
Three single-field use cases will be considered: single polarization (XX), parallel hands (XX, YY), and full polarization (XX, XY, YX, YY).  
Per-channel weight density is always `True`.

This notebook demonstrates only the serial core functions and does not include any parallelization.

These functions are all low level functions and not intended as the final external user interface. For example the functions expect all inputs in standard SI units and will not do any units conversion.

---

## How CASA Calculates Imaging Weights

The same weights are applied to all polarizations:

- **Single Polarization (XX):** Imaging weights are calculated using the XX data weights.  
- **Parallel Hands (XX, YY):** Imaging weights are calculated using the average of the XX and YY data weights.  
- **Full Polarization (XX, XY, YX, YY):** Imaging weights are calculated using the average of the XX and YY data weights and applied to all polarizations (XX, XY, YX, YY).  
  *Note: The XY and YX data weights are ignored.*

---

## Pseudo Code

ps_xdt Processing Set Data Tree
ms_xdt Measurement Set v4 Data Tree
```
core/imaging/calculate_imaging_weights(ps_xdt)
    for ms_xdt in ps_xdt
        #Grid the averaged data weights
        grid_imaging_weights(weight_density_grid,ms_xdt)

    #Calculate Briggs Parameters
    briggs_factors = calculate_briggs_params(weight_density_grid)
    
    #Degrid and applies Briggs parameter
    for ms_xdt in ps_xdt:
        imaging_weights = degrid_imaging_weights(weight_density_grid)
```

---

## Notes

- `mosweight` has not been implemented yet. This is planned for future work.  
- `perchanweightdensity` is always `True`.  
- `briggsbwtaper` has not been implemented yet.  
- CASA interpolation was changed from *linear* to *nearest*.  

---

## Questions

- Should we continue handling the weights in the same way as CASA, or should we consider a more sophisticated approach for the full polarization case?  
- Are there any function or parameter names we should change?  
- Are there missing weighting options that we should demonstrate?  
- What interpolation method should be used when data channel frequencies differ from image channel frequencies?  
- Should we calculate the weights independently for each execution block or scheduling block?  
  *Example: the last full polarization case (`3c286_Band6_5chans_lsrk_compare_weights.ps.zarr`) has three execution blocks from the same scheduling block.*

## Install AstroVIPER

Skip this cell if you don't want to install the latest version of AstroVIPER.

In [1]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

Using astroviper version 0.0.29


## Single Pol XX

In [2]:
from xradio.measurement_set import open_processing_set
from toolviper.utils.data import download, update
update()
download(file="twhya_selfcal_5chans_lsrk_xx_compare_weights.ps.zarr")
ps_single_pol_xdt = open_processing_set("twhya_selfcal_5chans_lsrk_xx_compare_weights.ps.zarr")
ps_single_pol_xdt.xr_ps.summary() 

[2025-09-16 12:00:52,149]     INFO    viperlog:  Updating file metadata information ...  


Output()

[2025-09-16 12:00:52,423]     INFO    viperlog:  Module path: /Users/jsteeb/Dropbox/toolviper/src/toolviper 
[2025-09-16 12:00:52,428]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List                                         
 ────────────────────────────────────────────────────── 
  twhya_selfcal_5chans_lsrk_xx_compare_weights.ps.zarr

[2025-09-16 12:00:52,430]     INFO    viperlog:  File exists: twhya_selfcal_5chans_lsrk_xx_compare_weights.ps.zarr 


Output()

,name,intents,shape,polarization,scan_name,spw_name,spw_intent,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,twhya_selfcal_5chans_lsrk_xx_0,[OBSERVE_TARGET#ON_SOURCE],"(270, 171, 5, 1)",[XX],"[12, 16, 20, 24, 28, 36]",ALMA_RB_07#BB_2#SW-01#FULL_RES_0,UNSPECIFIED,[TW Hya_5],[3c279_4],[],"[fk5, 11h01m51.80s, -34d42m17.37s]",3.727306e+11,3.727330e+11


In [3]:
ps_single_pol_xdt["twhya_selfcal_5chans_lsrk_xx_0"].frequency[1] - ps_single_pol_xdt["twhya_selfcal_5chans_lsrk_xx_0"].frequency[0]

<xarray.DataArray 'frequency' ()> Size: 8B
array(610330.15905762)

### Create Weights With AstroVIPER

In [4]:
from astroviper.core.imaging.calculate_imaging_weights import calculate_imaging_weights
from xradio.measurement_set import open_processing_set

import numpy as np

grid_params = {}
grid_params["image_size"] = [250, 250]
grid_params["cell_size"] = np.array([-0.1, 0.1]) * np.pi / (180 * 3600)



ms_single_pol_xdt = ps_single_pol_xdt["twhya_selfcal_5chans_lsrk_xx_0"]


for robust in [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]:
    print("Calculating robust =", robust)
    ps_single_pol_xdt, data_group_out = calculate_imaging_weights(
        ps_single_pol_xdt,
        grid_params=grid_params,
        imaging_weights_params={"weighting": "briggs", "robust": robust},
        sel_params={"overwrite": False, "data_group_out": {"weight_imaging":"WEIGHT_AV_R_" + str(robust)}, "data_group_out_name": "av" + str(robust)},
)

ps_single_pol_xdt["twhya_selfcal_5chans_lsrk_xx_0"]

Calculating robust = -2
Calculating robust = -1.45
Calculating robust = -0.5
Calculating robust = 0.0
Calculating robust = 0.5
Calculating robust = 1.33
Calculating robust = 2


<xarray.DataTree 'twhya_selfcal_5chans_lsrk_xx_0'>
Group: /twhya_selfcal_5chans_lsrk_xx_0
│   Dimensions:                     (time: 270, baseline_id: 171, frequency: 5,
│                                    polarization: 1, uvw_label: 3, bidxs: 44772,
│                                    row_id: 44772, tidxs: 44772)
│   Coordinates:
│       baseline_antenna1_name      (baseline_id) <U9 6kB ...
│       baseline_antenna2_name      (baseline_id) <U9 6kB ...
│     * baseline_id                 (baseline_id) int64 1kB 0 1 2 3 ... 168 169 170
│     * bidxs                       (bidxs) int64 358kB 0 1 2 3 ... 167 168 169 170
│       field_name                  (time) <U28 30kB ...
│     * frequency                   (frequency) float64 40B 3.727e+11 ... 3.727e+11
│     * polarization                (polarization) <U2 8B 'XX'
│     * row_id                      (row_id) int64 358kB 0 1 2 ... 44769 44770 44771
│       scan_name                   (time) <U21 23kB '12' '12' '12' ... '36' '36'
│     * tidxs                       (tidxs) int64 358kB 0 0 0 0 ... 269 269 269 269
│     * time                        (time) float64 2kB 1.353e+09 ... 1.353e+09
│     * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
│   Data variables: (12/20)
│       EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 369kB ...
│       FLAG                        (time, baseline_id, frequency, polarization) bool 231kB ...
│       TIME_CENTROID               (time, baseline_id) float64 369kB ...
│       UVW                         (time, baseline_id, uvw_label) float64 1MB 10...
│       VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 2MB ...
│       WEIGHT                      (time, baseline_id, frequency, polarization) float32 923kB ...
│       ...                          ...
│       WEIGHT_AV_R_-1.45           (time, baseline_id, frequency, polarization) float64 2MB ...
│       WEIGHT_AV_R_-0.5            (time, baseline_id, frequency, polarization) float64 2MB ...
│       WEIGHT_AV_R_0.0             (time, baseline_id, frequency, polarization) float64 2MB ...
│       WEIGHT_AV_R_0.5             (time, baseline_id, frequency, polarization) float64 2MB ...
│       WEIGHT_AV_R_1.33            (time, baseline_id, frequency, polarization) float64 2MB ...
│       WEIGHT_AV_R_2               (time, baseline_id, frequency, polarization) float64 2MB ...
│   Attributes:
│       creation_date:     2025-09-11T18:00:34.357485+00:00
│       creator:           {'software_name': 'xradio', 'version': '0.0.59'}
│       data_groups:       {'CASA_R_-0.5': {'correlated_data': 'VISIBILITY', 'dat...
│       observation_info:  {'intents': ['OBSERVE_TARGET#ON_SOURCE'], 'observer': ...
│       processor_info:    {'sub_type': 'ALMA_CORRELATOR_MODE', 'type': 'CORRELAT...
│       schema_version:    4.0.-9987
│       type:              visibility
├── Group: /twhya_selfcal_5chans_lsrk_xx_0/antenna_xds
│       Dimensions:                 (antenna_name: 19, cartesian_pos_label: 3,
│                                    receptor_label: 2)
│       Coordinates:
│         * antenna_name            (antenna_name) <U9 684B 'DA42_A050' ... 'DV23_A007'
│         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│           mount                   (antenna_name) <U6 456B ...
│           polarization_type       (antenna_name, receptor_label) <U1 152B ...
│         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│           station_name            (antenna_name) <U4 304B ...
│           telescope_name          (antenna_name) <U4 304B ...
│       Data variables:
│           ANTENNA_DISH_DIAMETER   (antenna_name) float64 152B ...
│           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 456B ...
│           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 304B ...
│       Attributes:
│           overall_telescope_name:  ALMA
│           relocatable_antennas:    True
│           type

## Plotting code

In [5]:
def plot_astroviper_vs_casa_weights_interactive(
    ps_xdt, ms_xdt_name, robust, freq_idx, bl_idx
):
    import matplotlib.pyplot as plt
    import numpy as np

    ms_xdt = ps_xdt[ms_xdt_name]
    av = ms_xdt[f"WEIGHT_AV_R_{robust}"].isel(
        baseline_id=bl_idx, frequency=freq_idx, polarization=0
    )
    casa = ms_xdt[f"WEIGHT_CASA_R_{robust}"].isel(
        baseline_id=bl_idx, frequency=freq_idx
    )

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    ax1.plot(av, label="AstroVIPER WEIGHT")
    ax1.plot(casa, label="CASA WEIGHT")

    if robust == 2:
        natural_weights = ms_xdt[f"WEIGHT"].isel(baseline_id=bl_idx, frequency=freq_idx)
        ax1.plot(natural_weights, label="Natural WEIGHT", linestyle="--")

    ax1.set(
        ylabel="Imaging Weight",
        title=f"R={robust} | freq={freq_idx} | baseline={bl_idx}",
    )

    per_dif = 100 * (av - casa) / (casa.max())
    ax2.plot(per_dif, label="Percentage Difference (AstroVIPER - CASA)")
    ax2.set(xlabel="Time Index", ylabel="Percentage Difference")

    print("The max difference is:", np.nanmax(per_dif))

    ax1.grid(True)
    ax1.legend()
    plt.show()
    plt.close(fig)


def plot_astroviper_vs_casa_weights_imshow_interactive(
    ps_xdt, ms_xdt_name, robust, bl_idx
):
    import matplotlib.pyplot as plt
    import numpy as np

    ms_xdt = ps_xdt[ms_xdt_name]
    av = ms_xdt[f"WEIGHT_AV_R_{robust}"].isel(baseline_id=bl_idx, polarization=0).values
    casa = ms_xdt[f"WEIGHT_CASA_R_{robust}"].isel(baseline_id=bl_idx).values
    av[av == 0] = np.nan
    casa[casa == 0] = np.nan

    per_dif = 100 * (av - casa) / (np.nanmax(casa))

    # Wide figure for horizontal layout
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8), constrained_layout=True)

    im_kwargs = dict(aspect="auto", interpolation="nearest")

    # Plot AstroVIPER
    im1 = ax1.imshow(av, **im_kwargs)
    ax1.set_box_aspect(1)
    ax1.set_title(f"AstroVIPER | R={robust} | baseline={bl_idx}")
    ax1.set_ylabel("Imaging Weight")
    ax1.set_xlabel("Frequency Index")
    ax1.set_ylabel("Time Index")
    fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)

    # Plot CASA
    im2 = ax2.imshow(casa, **im_kwargs)
    ax2.set_box_aspect(1)
    ax2.set_title(f"CASA | R={robust} | baseline={bl_idx}")
    ax2.set_ylabel("Imaging Weight")
    ax2.set_xlabel("Frequency Index")
    ax2.set_ylabel("Time Index")
    fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)

    # Plot % difference
    im3 = ax3.imshow(per_dif, **im_kwargs)
    ax3.set_box_aspect(1)
    ax3.set_title("Percentage Difference (AstroVIPER - CASA)")
    ax3.set_xlabel("Frequency Index")
    ax3.set_ylabel("Time Index")
    fig.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)

    print("The max difference is:", float(np.nanmax(per_dif)))

    plt.show()
    plt.close(fig)

### Compare CASA and AstroVIPER

CASA flags the first channel.

In [6]:
# If not installed:
# !pip install ipywidgets
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]

interact(
    plot_astroviper_vs_casa_weights_interactive,
    ps_xdt=fixed(ps_single_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_single_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    freq_idx=widgets.IntSlider(min=0, max=4, step=1, value=2, description="freq_chan"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('twhya_selfcal_5chans_lsrk_xx_0',), value='…

<function __main__.plot_astroviper_vs_casa_weights_interactive(ps_xdt, ms_xdt_name, robust, freq_idx, bl_idx)>

In [7]:
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]

interact(
    plot_astroviper_vs_casa_weights_imshow_interactive,
    ps_xdt=fixed(ps_single_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_single_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('twhya_selfcal_5chans_lsrk_xx_0',), value='…

<function __main__.plot_astroviper_vs_casa_weights_imshow_interactive(ps_xdt, ms_xdt_name, robust, bl_idx)>

## Dual Pol XX, YY

In [8]:
from xradio.measurement_set import open_processing_set
from toolviper.utils.data import download, update
update()
download(file="twhya_selfcal_5chans_lsrk_compare_weights.ps.zarr")
ps_dual_pol_xdt = open_processing_set("twhya_selfcal_5chans_lsrk_compare_weights.ps.zarr")
ps_dual_pol_xdt.xr_ps.summary()

[2025-09-16 12:00:54,194]     INFO    viperlog:  Updating file metadata information ...  


Output()

[2025-09-16 12:00:54,376]     INFO    viperlog:  Module path: /Users/jsteeb/Dropbox/toolviper/src/toolviper 
[2025-09-16 12:00:54,378]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List                                      
 ─────────────────────────────────────────────────── 
  twhya_selfcal_5chans_lsrk_compare_weights.ps.zarr

[2025-09-16 12:00:54,380]     INFO    viperlog:  File exists: twhya_selfcal_5chans_lsrk_compare_weights.ps.zarr 


Output()

,name,intents,shape,polarization,scan_name,spw_name,spw_intent,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,twhya_selfcal_5chans_lsrk_0,[OBSERVE_TARGET#ON_SOURCE],"(270, 171, 5, 2)","[XX, YY]","[12, 16, 20, 24, 28, 36]",ALMA_RB_07#BB_2#SW-01#FULL_RES_0,UNSPECIFIED,[TW Hya_5],[3c279_4],[],"[fk5, 11h01m51.80s, -34d42m17.37s]",3.727306e+11,3.727330e+11


In [9]:
ps_dual_pol_xdt["twhya_selfcal_5chans_lsrk_0"]



<xarray.DataTree 'twhya_selfcal_5chans_lsrk_0'>
Group: /twhya_selfcal_5chans_lsrk_0
│   Dimensions:                     (time: 270, baseline_id: 171, frequency: 5,
│                                    polarization: 2, uvw_label: 3, bidxs: 44772,
│                                    row_id: 44772, tidxs: 44772)
│   Coordinates:
│       baseline_antenna1_name      (baseline_id) <U9 6kB ...
│       baseline_antenna2_name      (baseline_id) <U9 6kB ...
│     * baseline_id                 (baseline_id) int64 1kB 0 1 2 3 ... 168 169 170
│     * bidxs                       (bidxs) int64 358kB 0 1 2 3 ... 167 168 169 170
│       field_name                  (time) <U28 30kB ...
│     * frequency                   (frequency) float64 40B 3.727e+11 ... 3.727e+11
│     * polarization                (polarization) <U2 16B 'XX' 'YY'
│     * row_id                      (row_id) int64 358kB 0 1 2 ... 44769 44770 44771
│       scan_name                   (time) <U21 23kB '12' '12' '12' ... '36' '36'
│     * tidxs                       (tidxs) int64 358kB 0 0 0 0 ... 269 269 269 269
│     * time                        (time) float64 2kB 1.353e+09 ... 1.353e+09
│     * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
│   Data variables: (12/13)
│       EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 369kB ...
│       FLAG                        (time, baseline_id, frequency, polarization) bool 462kB ...
│       TIME_CENTROID               (time, baseline_id) float64 369kB ...
│       UVW                         (time, baseline_id, uvw_label) float64 1MB ...
│       VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 4MB ...
│       WEIGHT                      (time, baseline_id, frequency, polarization) float32 2MB ...
│       ...                          ...
│       WEIGHT_CASA_R_-1.45         (time, baseline_id, frequency) float64 2MB ...
│       WEIGHT_CASA_R_-2            (time, baseline_id, frequency) float64 2MB ...
│       WEIGHT_CASA_R_0             (time, baseline_id, frequency) float64 2MB ...
│       WEIGHT_CASA_R_0.5           (time, baseline_id, frequency) float64 2MB ...
│       WEIGHT_CASA_R_1.33          (time, baseline_id, frequency) float64 2MB ...
│       WEIGHT_CASA_R_2             (time, baseline_id, frequency) float64 2MB ...
│   Attributes:
│       creation_date:     2025-09-11T16:50:05.578634+00:00
│       creator:           {'software_name': 'xradio', 'version': '0.0.59'}
│       data_groups:       {'CASA_R_-0.5': {'correlated_data': 'VISIBILITY', 'dat...
│       observation_info:  {'intents': ['OBSERVE_TARGET#ON_SOURCE'], 'observer': ...
│       processor_info:    {'sub_type': 'ALMA_CORRELATOR_MODE', 'type': 'CORRELAT...
│       schema_version:    4.0.-9987
│       type:              visibility
├── Group: /twhya_selfcal_5chans_lsrk_0/antenna_xds
│       Dimensions:                 (antenna_name: 19, cartesian_pos_label: 3,
│                                    receptor_label: 2)
│       Coordinates:
│         * antenna_name            (antenna_name) <U9 684B 'DA42_A050' ... 'DV23_A007'
│         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│           mount                   (antenna_name) <U6 456B ...
│           polarization_type       (antenna_name, receptor_label) <U1 152B ...
│         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│           station_name            (antenna_name) <U4 304B ...
│           telescope_name          (antenna_name) <U4 304B ...
│       Data variables:
│           ANTENNA_DISH_DIAMETER   (antenna_name) float64 152B ...
│           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 456B ...
│           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 304B ...
│       Attributes:
│           overall_telescope_name:  ALMA
│           relocatable_antennas:    True
│           type:                    antenna
├── Group: /twhya_selfcal_5chans_lsrk_0/field_and_source_base_

### Create Weights With AstroVIPER

In [10]:
from astroviper.core.imaging.calculate_imaging_weights import calculate_imaging_weights
from xradio.measurement_set import open_processing_set

import numpy as np

grid_params = {}
grid_params["image_size"] = [250, 250]
grid_params["cell_size"] = np.array([-0.1, 0.1]) * np.pi / (180 * 3600)

for robust in [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]:
    print("Calculating robust =", robust)
    ps_dual_pol_xdt, data_group_out = calculate_imaging_weights(
        ps_dual_pol_xdt,
        grid_params=grid_params,
        imaging_weights_params={"weighting": "briggs", "robust": robust},
        sel_params={"overwrite": False, "data_group_out": {"weight_imaging":"WEIGHT_AV_R_" + str(robust)}, "data_group_out_name": "av" + str(robust)},
)

ps_dual_pol_xdt

Calculating robust = -2
Calculating robust = -1.45
Calculating robust = -0.5
Calculating robust = 0.0
Calculating robust = 0.5
Calculating robust = 1.33
Calculating robust = 2


<xarray.DataTree>
Group: /
│   Attributes:
│       type:     processing_set
└── Group: /twhya_selfcal_5chans_lsrk_0
    │   Dimensions:                     (time: 270, baseline_id: 171, frequency: 5,
    │                                    polarization: 2, uvw_label: 3, bidxs: 44772,
    │                                    row_id: 44772, tidxs: 44772)
    │   Coordinates:
    │       baseline_antenna1_name      (baseline_id) <U9 6kB ...
    │       baseline_antenna2_name      (baseline_id) <U9 6kB ...
    │     * baseline_id                 (baseline_id) int64 1kB 0 1 2 3 ... 168 169 170
    │     * bidxs                       (bidxs) int64 358kB 0 1 2 3 ... 167 168 169 170
    │       field_name                  (time) <U28 30kB ...
    │     * frequency                   (frequency) float64 40B 3.727e+11 ... 3.727e+11
    │     * polarization                (polarization) <U2 16B 'XX' 'YY'
    │     * row_id                      (row_id) int64 358kB 0 1 2 ... 44769 44770 44771
    │       scan_name                   (time) <U21 23kB '12' '12' '12' ... '36' '36'
    │     * tidxs                       (tidxs) int64 358kB 0 0 0 0 ... 269 269 269 269
    │     * time                        (time) float64 2kB 1.353e+09 ... 1.353e+09
    │     * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
    │   Data variables: (12/20)
    │       EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 369kB ...
    │       FLAG                        (time, baseline_id, frequency, polarization) bool 462kB ...
    │       TIME_CENTROID               (time, baseline_id) float64 369kB ...
    │       UVW                         (time, baseline_id, uvw_label) float64 1MB 10...
    │       VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 4MB ...
    │       WEIGHT                      (time, baseline_id, frequency, polarization) float32 2MB ...
    │       ...                          ...
    │       WEIGHT_AV_R_-1.45           (time, baseline_id, frequency, polarization) float64 4MB ...
    │       WEIGHT_AV_R_-0.5            (time, baseline_id, frequency, polarization) float64 4MB ...
    │       WEIGHT_AV_R_0.0             (time, baseline_id, frequency, polarization) float64 4MB ...
    │       WEIGHT_AV_R_0.5             (time, baseline_id, frequency, polarization) float64 4MB ...
    │       WEIGHT_AV_R_1.33            (time, baseline_id, frequency, polarization) float64 4MB ...
    │       WEIGHT_AV_R_2               (time, baseline_id, frequency, polarization) float64 4MB ...
    │   Attributes:
    │       creation_date:     2025-09-11T16:50:05.578634+00:00
    │       creator:           {'software_name': 'xradio', 'version': '0.0.59'}
    │       data_groups:       {'CASA_R_-0.5': {'correlated_data': 'VISIBILITY', 'dat...
    │       observation_info:  {'intents': ['OBSERVE_TARGET#ON_SOURCE'], 'observer': ...
    │       processor_info:    {'sub_type': 'ALMA_CORRELATOR_MODE', 'type': 'CORRELAT...
    │       schema_version:    4.0.-9987
    │       type:              visibility
    ├── Group: /twhya_selfcal_5chans_lsrk_0/antenna_xds
    │       Dimensions:                 (antenna_name: 19, cartesian_pos_label: 3,
    │                                    receptor_label: 2)
    │       Coordinates:
    │         * antenna_name            (antenna_name) <U9 684B 'DA42_A050' ... 'DV23_A007'
    │         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    │           mount                   (antenna_name) <U6 456B ...
    │           polarization_type       (antenna_name, receptor_label) <U1 152B ...
    │         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
    │           station_name            (antenna_name) <U4 304B ...
    │           telescope_name          (antenna_name) <U4 304B ...
    │       Data variables:
    │           ANTENNA_DISH_DIAMETER   (antenna_name) float64 152B ...
    │           ANTENNA_POSITION        (antenna_name, cartesi

In [11]:
# If not installed:
# !pip install ipywidgets
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]
    
ms_dual_pol_xdt = ps_dual_pol_xdt["twhya_selfcal_5chans_lsrk_0"]

interact(
    plot_astroviper_vs_casa_weights_interactive,
    ps_xdt=fixed(ps_dual_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_dual_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    freq_idx=widgets.IntSlider(min=0, max=4, step=1, value=2, description="freq_chan"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('twhya_selfcal_5chans_lsrk_0',), value='twh…

<function __main__.plot_astroviper_vs_casa_weights_interactive(ps_xdt, ms_xdt_name, robust, freq_idx, bl_idx)>

In [12]:
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]

ms_dual_pol_xdt = ps_dual_pol_xdt["twhya_selfcal_5chans_lsrk_0"]

interact(
    plot_astroviper_vs_casa_weights_imshow_interactive,
    ps_xdt=fixed(ps_dual_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_dual_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('twhya_selfcal_5chans_lsrk_0',), value='twh…

<function __main__.plot_astroviper_vs_casa_weights_imshow_interactive(ps_xdt, ms_xdt_name, robust, bl_idx)>

## Full Pol XX, XY, YX, YY

In [13]:
from xradio.measurement_set import open_processing_set
from toolviper.utils.data import download, update
update()
download(file="3c286_Band6_5chans_lsrk_compare_weights.ps.zarr")

ps_full_pol_xdt = open_processing_set("3c286_Band6_5chans_lsrk_compare_weights.ps.zarr")
ps_full_pol_xdt.xr_ps.summary()

[2025-09-16 12:00:55,353]     INFO    viperlog:  Updating file metadata information ...  


Output()

[2025-09-16 12:00:55,548]     INFO    viperlog:  Module path: /Users/jsteeb/Dropbox/toolviper/src/toolviper 
[2025-09-16 12:00:55,548]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List                                    
 ───────────────────────────────────────────────── 
  3c286_Band6_5chans_lsrk_compare_weights.ps.zarr

[2025-09-16 12:00:55,550]     INFO    viperlog:  File exists: 3c286_Band6_5chans_lsrk_compare_weights.ps.zarr 


Output()

,name,intents,shape,polarization,scan_name,spw_name,spw_intent,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,3c286_Band6_5chans_lsrk_0,[OBSERVE_TARGET#ON_SOURCE],"(147, 212, 5, 4)","[XX, XY, YX, YY]","[12, 14, 17, 25]",ALMA_RB_06#BB_3#SW-01#FULL_RES_0,UNSPECIFIED,[3c286_0],[3c286_0],[Single_Continuum(ID=0)],"[fk5, 13h31m08.29s, 30d30m32.96s]",2.398226e+11,2.403227e+11
1,3c286_Band6_5chans_lsrk_1,[OBSERVE_TARGET#ON_SOURCE],"(147, 212, 5, 4)","[XX, XY, YX, YY]","[37, 39, 41, 44, 52, 54, 56]",ALMA_RB_06#BB_3#SW-01#FULL_RES_0,UNSPECIFIED,[3c286_0],[3c286_0],[Single_Continuum(ID=0)],"[fk5, 13h31m08.29s, 30d30m32.96s]",2.398226e+11,2.403227e+11
2,3c286_Band6_5chans_lsrk_2,[OBSERVE_TARGET#ON_SOURCE],"(147, 212, 5, 4)","[XX, XY, YX, YY]","[69, 71, 73, 76, 84, 86, 88]",ALMA_RB_06#BB_3#SW-01#FULL_RES_0,UNSPECIFIED,[3c286_0],[3c286_0],[Single_Continuum(ID=0)],"[fk5, 13h31m08.29s, 30d30m32.96s]",2.398226e+11,2.403227e+11


In [14]:
ps_full_pol_xdt["3c286_Band6_5chans_lsrk_0"]

<xarray.DataTree '3c286_Band6_5chans_lsrk_0'>
Group: /3c286_Band6_5chans_lsrk_0
│   Dimensions:                     (time: 147, baseline_id: 212, frequency: 5,
│                                    polarization: 4, uvw_label: 3, bidxs: 31164,
│                                    row_id: 31164, tidxs: 31164)
│   Coordinates:
│       baseline_antenna1_name      (baseline_id) <U9 8kB ...
│       baseline_antenna2_name      (baseline_id) <U9 8kB ...
│     * baseline_id                 (baseline_id) int64 2kB 0 1 2 3 ... 209 210 211
│     * bidxs                       (bidxs) int64 249kB 0 1 2 3 ... 208 209 210 211
│       field_name                  (time) <U27 16kB ...
│     * frequency                   (frequency) float64 40B 2.398e+11 ... 2.403e+11
│     * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
│     * row_id                      (row_id) int64 249kB 0 1 2 ... 31161 31162 31163
│       scan_name                   (time) <U21 12kB '12' '12' '12' ... '25' '25'
│     * tidxs                       (tidxs) int64 249kB 0 0 0 0 ... 146 146 146 146
│     * time                        (time) float64 1kB 1.404e+09 ... 1.404e+09
│     * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
│   Data variables: (12/13)
│       EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 249kB ...
│       FLAG                        (time, baseline_id, frequency, polarization) bool 623kB ...
│       TIME_CENTROID               (time, baseline_id) float64 249kB ...
│       UVW                         (time, baseline_id, uvw_label) float64 748kB ...
│       VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 5MB ...
│       WEIGHT                      (time, baseline_id, frequency, polarization) float32 2MB ...
│       ...                          ...
│       WEIGHT_CASA_R_-1.45         (time, baseline_id, frequency) float64 1MB ...
│       WEIGHT_CASA_R_-2            (time, baseline_id, frequency) float64 1MB ...
│       WEIGHT_CASA_R_0             (time, baseline_id, frequency) float64 1MB ...
│       WEIGHT_CASA_R_0.5           (time, baseline_id, frequency) float64 1MB ...
│       WEIGHT_CASA_R_1.33          (time, baseline_id, frequency) float64 1MB ...
│       WEIGHT_CASA_R_2             (time, baseline_id, frequency) float64 1MB ...
│   Attributes:
│       creation_date:     2025-09-11T16:50:06.207363+00:00
│       creator:           {'software_name': 'xradio', 'version': '0.0.59'}
│       data_groups:       {'CASA_R_-0.5': {'correlated_data': 'VISIBILITY', 'dat...
│       observation_info:  {'execution_block_UID': '<EntityRef entityId="uid://A0...
│       processor_info:    {'sub_type': 'ALMA_BASELINE', 'type': 'CORRELATOR'}
│       schema_version:    4.0.-9987
│       type:              visibility
├── Group: /3c286_Band6_5chans_lsrk_0/antenna_xds
│       Dimensions:                 (antenna_name: 31, cartesian_pos_label: 3,
│                                    receptor_label: 2)
│       Coordinates:
│         * antenna_name            (antenna_name) <U9 1kB 'DA41_A079' ... 'DV25_A056'
│         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│           mount                   (antenna_name) <U6 744B ...
│           polarization_type       (antenna_name, receptor_label) <U1 248B ...
│         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│           station_name            (antenna_name) <U4 496B ...
│           telescope_name          (antenna_name) <U4 496B ...
│       Data variables:
│           ANTENNA_DISH_DIAMETER   (antenna_name) float64 248B ...
│           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 744B ...
│           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 496B ...
│       Attributes:
│           overall_telescope_name:  ALMA
│           relocatable_antennas:    True
│           type:                    antenna
├── Group: /3c286_Band6_5chans_lsrk_0/field_and_source_base_xds

### Create Weights With AstroVIPER

In [15]:
from astroviper.core.imaging.calculate_imaging_weights import calculate_imaging_weights
from xradio.measurement_set import open_processing_set

import numpy as np

grid_params = {}
grid_params["image_size"] = [250, 250]
grid_params["cell_size"] = np.array([-0.1, 0.1]) * np.pi / (180 * 3600)

for robust in [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]:
    print("Calculating robust =", robust)
    ps_full_pol_xdt, data_group_out = calculate_imaging_weights(
        ps_full_pol_xdt,
        grid_params=grid_params,
        imaging_weights_params={"weighting": "briggs", "robust": robust},
        sel_params={"overwrite": False, "data_group_out": {"weight_imaging":"WEIGHT_AV_R_" + str(robust)}, "data_group_out_name": "av" + str(robust)},
)

ms_full_pol_xdt = ps_full_pol_xdt["3c286_Band6_5chans_lsrk_0"]

Calculating robust = -2
Calculating robust = -1.45
Calculating robust = -0.5
Calculating robust = 0.0
Calculating robust = 0.5
Calculating robust = 1.33
Calculating robust = 2


In [16]:

ms_full_pol_xdt["WEIGHT_AV_R_-2"]

<xarray.DataArray 'WEIGHT_AV_R_-2' (time: 147, baseline_id: 212, frequency: 5,
                                    polarization: 4)> Size: 5MB
array([[[[ 0.84904365,  0.84904365,  0.84904365,  0.84904365],
         [ 1.14857846,  1.14857846,  1.14857846,  1.14857846],
         [ 1.12285032,  1.12285032,  1.12285032,  1.12285032],
         [ 1.74006389,  1.74006389,  1.74006389,  1.74006389],
         [        nan,         nan,         nan,         nan]],

        [[ 0.84904356,  0.84904356,  0.84904356,  0.84904356],
         [16.87678421, 16.87678421, 16.87678421, 16.87678421],
         [14.74758723, 14.74758723, 14.74758723, 14.74758723],
         [13.09533055, 13.09533055, 13.09533055, 13.09533055],
         [        nan,         nan,         nan,         nan]],

        [[ 0.84904369,  0.84904369,  0.84904369,  0.84904369],
         [ 5.63277302,  5.63277302,  5.63277302,  5.63277302],
         [ 5.36972449,  5.36972449,  5.36972449,  5.36972449],
         [ 4.91761864,  4.91761864,  4.91761864,  4.91761864],
         [        nan,         nan,         nan,         nan]],

        ...,

...
        ...,

        [[        nan,         nan,         nan,         nan],
         [ 0.38498086,  0.38498086,  0.38498086,  0.38498086],
         [ 0.38203322,  0.38203322,  0.38203322,  0.38203322],
         [ 0.3817664 ,  0.3817664 ,  0.3817664 ,  0.3817664 ],
         [ 0.37544553,  0.37544553,  0.37544553,  0.37544553]],

        [[        nan,         nan,         nan,         nan],
         [ 0.84968398,  0.84968398,  0.84968398,  0.84968398],
         [ 0.85525789,  0.85525789,  0.85525789,  0.85525789],
         [ 0.85432592,  0.85432592,  0.85432592,  0.85432592],
         [ 0.85783304,  0.85783304,  0.85783304,  0.85783304]],

        [[        nan,         nan,         nan,         nan],
         [ 0.49384327,  0.49384327,  0.49384327,  0.49384327],
         [ 0.49312108,  0.49312108,  0.49312108,  0.49312108],
         [ 0.49254147,  0.49254147,  0.49254147,  0.49254147],
         [ 0.4937827 ,  0.4937827 ,  0.4937827 ,  0.4937827 ]]]],
      shape=(147, 212, 5, 4))
Coordinates:
    baseline_antenna1_name  (baseline_id) <U9 8kB ...
    baseline_antenna2_name  (baseline_id) <U9 8kB ...
  * baseline_id             (baseline_id) int64 2kB 0 1 2 3 ... 208 209 210 211
    field_name              (time) <U27 16kB ...
  * frequency               (frequency) float64 40B 2.398e+11 ... 2.403e+11
  * polarization            (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
    scan_name               (time) <U21 12kB '12' '12' '12' ... '25' '25' '25'
  * time                    (time) float64 1kB 1.404e+09 1.404e+09 ... 1.404e+09

In [17]:
# If not installed:
# !pip install ipywidgets
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]


interact(
    plot_astroviper_vs_casa_weights_interactive,
    ps_xdt=fixed(ps_full_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_full_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    freq_idx=widgets.IntSlider(min=0, max=4, step=1, value=2, description="freq_chan"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('3c286_Band6_5chans_lsrk_0', '3c286_Band6_5…

<function __main__.plot_astroviper_vs_casa_weights_interactive(ps_xdt, ms_xdt_name, robust, freq_idx, bl_idx)>

In [18]:
# %matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, fixed

robust_options = [-2, -1.45, -0.5, 0.0, 0.5, 1.33, 2]

interact(
    plot_astroviper_vs_casa_weights_imshow_interactive,
    ps_xdt=fixed(ps_full_pol_xdt),
    ms_xdt_name=widgets.Dropdown(options=list(ps_full_pol_xdt.keys())),
    robust=widgets.Dropdown(options=robust_options, value=-2, description="robust"),
    bl_idx=widgets.IntSlider(min=0, max=170, step=1, value=10, description="baseline_id"),
)

interactive(children=(Dropdown(description='ms_xdt_name', options=('3c286_Band6_5chans_lsrk_0', '3c286_Band6_5…

<function __main__.plot_astroviper_vs_casa_weights_imshow_interactive(ps_xdt, ms_xdt_name, robust, bl_idx)>